# GTFS Onboarding Exercise

## Introduction

The [MBTA’s GTFS Feed](https://github.com/mbta/gtfs-documentation/blob/master/reference/gtfs.md) consists of a series of [text files in a standard format](https://developers.google.com/transit/gtfs/reference) that represent the system’s static schedule (along with other geographic information). This data ultimately underlies the schedule data available through the [MBTA's V3 API](https://www.mbta.com/developers/v3-api), and is referenced by data included in our [GTFS-realtime](https://www.mbta.com/developers/gtfs-realtime) feeds (not covered in this guide).

Throughout this guide, you will gain familiarity with the data tables included in the feed, organized by their corresponding API resources. For the purposes of this guide, we will be using the GTFS Feed active on [October 14, 2020](https://cdn.mbta.com/archive/archived_feeds.txt).

In the following sections, you will cover the these concepts:
* [Services](#Services): A set of dates on which trips run for one or more routes
* [Routes](#Routes): Data about individual routes
* [Route Patterns](#Route-Patterns): Subsets of a route, representing different possible patterns of where trips may serve
* [Trips](#Trips): The journey of a particular vehicle through a set of stops
* [Schedules](#Schedules): When a vehicle should arrive and depart a given stop for a given trip
* [Stops](#Stops): Details for a specific stop

At the end of each section, there will be a question to test your understanding. If you'd like to see an example solution, click the ellipsis under "Reveal Solution"

![](img/reveal.png)

To open a table of contents, click this button in the sidebar.

![](img/toc.png)

### Resources
* [GTFS Documentation](https://github.com/google/transit/blob/master/gtfs/spec/en/reference.md)
* [MBTA GTFS Documentation](https://github.com/mbta/gtfs-documentation/blob/master/reference/gtfs.md)
* [MBTA V3 API Documentation](https://api-v3.mbta.com/docs/swagger/index.html)

## Getting Oriented

### Jupyter Notebook
You can simply follow along with this guide and run the sample code by selecting code cells and pressing the "Run" button in the toolbar or pressing `Shift + Return`. To learn more about the Jupyter Notebook and its ecoystem, see its [documentation](https://jupyter.org/documentation).

### SQL
A SQLite database with tables corresponding to each table in the [MBTA GTFS Documenation](https://github.com/mbta/gtfs-documentation/blob/master/reference/gtfs.md) has been loaded for your convenience. Data exploration will be performed in SQL throughout this guide. 

In [1]:
.open feed.db

You can now execute SQL queries on the database.

In [37]:
-- Select this cell and press Shift + Return

SELECT *
FROM
	feed_info;

feed_publisher_name,feed_publisher_url,feed_lang,feed_start_date,feed_end_date,feed_version,feed_contact_email
MBTA,http://www.mbta.com,EN,20201002,20201007,"Fall 2020, 2020-10-09T17:32:32+00:00, version D",developer@mbta.com


**Note**: While SQL is convenient to use for exploring tabular data like the GTFS Feed, it's important to note that it is not required. In fact, SQL is not _actually_ used anywhere for creating or querying our GTFS feed in our various systems. You should feel free to learn the underlying concepts and explore the data using any tool with which you feel comfortable.

### Raw Data
The raw `.txt` file are available to download or inspect in the `feed` directory. For example, see [/feed/feed_info.txt](feed/feed_info.txt). Feel free to explore these files in Excel, Google Sheets, or your favorite spreadsheet software.

## Services
*A set of dates on which trips run for one or more routes*

**GTFS Tables**: [`calendar`](feed/calendar.txt), [`calendar_dates`](feed/calendar_dates.txt), [`calendar_attributes`](feed/calendar_attributes.txt)

**MBTA V3 API Resource**: [`Service`](https://api-v3.mbta.com/docs/swagger/index.html#/Service/ApiWeb_ServiceController_index)

When a rider is trying to figure out if and when a particular route is running trips, they probably have to ask themselves a few questions:
* What is the day of week?
* What's the date?
* Is it a holiday?

As an example, let's look at the 89 bus using MBTA.com's [Schedule Finder](https://www.mbta.com/schedules/89/line?schedule_direction%5Bdirection_id%5D=0&schedule_direction%5Borigin%5D=place-sull), or the [PDF schedule](https://cdn.mbta.com/sites/default/files/route_pdfs/2020-fall/R089.pdf). On MBTA.com, note the four different schedules to choose from (pictured below):
* Weekday Schedule (Fall)
* Saturday Schedule (Fall)
* Sunday Schedule (Fall)
* Thanksgiving Day, Nov 26

![](img/services.png)

As you can see in the Schedule Finder and PDF schedules, the major difference between these services is in the number and frequency of scheduled trips, though in some cases the pattern, or combination of stops, can also vary. If you've ever heard someone refer to the <q>Fall weekday schedule</q> or a <q>Holiday schedule,</q> they are referring to this concept of a service.

So how do we know on which dates these services are active and running trips, and how is this represented in our GTFS data? Through a combination of the data in the `calendar`, `calendar_attributes`, and `calendar_dates` tables.

### `calendar`
First, let's take a look at the first few rows of the `calendar` table.

In [3]:
SELECT *
FROM
	calendar
LIMIT
	3;

service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
BUS420-10-Wdy-02,1,1,1,1,1,0,0,20201002,20201218
BUS420-11-Sa-02,0,0,0,0,0,1,0,20201003,20201219
BUS420-12-Su-02,0,0,0,0,0,0,1,20201004,20201213


The `service_id` is just a unique identifier for a service.

For each column representing a day of week (`monday`, `tuesday`, etc.), a value of `1` indicates that the service **does** run trips on that day of week, and a value of `0` indicates that the service **does not** run trips on that day of week.

The `start_date` and `end_date` fields indicate the date range for which the service is active. 

So looking at service `BUS420-10-Wdy-02`, we can say it runs trips on weekdays only from October 2, 2020 to December 18, 2020.

So if we wanted to find services running trips only on weekdays, we could run this query:

In [4]:
SELECT *
FROM
	calendar
WHERE
	monday
		= 1
	AND tuesday
		= 1
	AND wednesday
		= 1
	AND thursday
		= 1
	AND friday
		= 1
	AND saturday
		= 0
	AND sunday
		= 0
ORDER BY
	service_id
LIMIT
	3;

service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
BUS420-10-Wdy-02,1,1,1,1,1,0,0,20201002,20201218
Boat-F1-WkdLmd,1,1,1,1,1,0,0,20201002,20201218
Boat-F4-Wkd,1,1,1,1,1,0,0,20201002,20201218


And if we want to find services ending before December 1, 2020, we could run this query:

In [5]:
SELECT *
FROM
	calendar
WHERE
	end_date
		< '20201201'
ORDER BY
	end_date
LIMIT
	3;

service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
RTL420-5-Wdy-01-OrgSkpNrhStnNrd,0,0,0,0,1,0,0,20201002,20201002
LRV420-2-Sa-01-CldCrlClgCrr,0,0,0,0,0,1,0,20201003,20201003
RTL420-6-Sa-01-OkGrvWlnOrgSkpNrhStnNrd,0,0,0,0,0,1,0,20201003,20201003


### `calendar_attributes`
We include additional data about services in the `calendar_attributes` table. Let's take a look at the first few rows.

In [6]:
SELECT *
FROM
	calendar_attributes
LIMIT
	3;

service_id,service_description,service_schedule_name,service_schedule_type,service_schedule_typicality,rating_start_date,rating_end_date,rating_description
BUS420-10-Wdy-02,Weekday schedule,Weekday,Weekday,1,20200830,20201219,Fall
BUS420-11-Sa-02,Saturday schedule,Saturday,Saturday,1,20200830,20201219,Fall
BUS420-12-Su-02,Sunday schedule,Sunday,Sunday,1,20200830,20201219,Fall


As you can see, this table includes a `service_id` column, which corresponds to a row in the `calendar` table, so we can combine these two tables to view human readable names and descriptions for each service.

In [7]:
SELECT
	calendar.service_id,
	calendar_attributes.service_schedule_name,
	calendar_attributes.service_description
FROM
	calendar

JOIN
	calendar_attributes
	ON
		calendar.service_id = calendar_attributes.service_id

LIMIT
	3;

service_id,service_schedule_name,service_description
BUS420-10-Wdy-02,Weekday,Weekday schedule
BUS420-11-Sa-02,Saturday,Saturday schedule
BUS420-12-Su-02,Sunday,Sunday schedule


### `calendar_dates`
Rows in the `calendar_dates` table define exceptions to the default service patterns defined in the `calendar` table. Let's take a look at the first few rows.

In [8]:
SELECT *
FROM
	calendar_dates
LIMIT
	10;

service_id,date,exception_type,holiday_name
LRV420-1-Wdy-01,20201002,1,
RTL420-9-Wdy-01,20201002,1,
LRV420-1-Wdy-01,20201009,1,
RTL420-9-Wdy-01,20201009,1,
BUS420-10-Wdy-02,20201012,2,Columbus Day
BUS420-W-Wdy-02,20201012,1,Columbus Day
ColumbusDay-VeteransDay-DayafterThanksgivingDay,20201012,1,Columbus Day
FallWeekday,20201012,2,Columbus Day
RTL420-5-Wdy-01,20201016,2,
RTL420-5-Wdy-01-OrgSkpNrhStnNrd,20201016,1,


Like `calendar_attributes`, the `service_id` in `calendar_dates` corresponds to a row in the `calendar` table. 

An `exception_type` of `1` indicates that we have added the service and its trips on the specified `date`.

An `exception_type` of `2` indicates that we have removed the service and its trips on the specified `date`.

As seen above, on October 16, 2020, we added service `RTL420-5-Wdy-01-OrgSkpNrhStnNrd` and removed service `RTL420-5-Wdy-01`.

### Test Your Knowledge

#### What is the `service_id`, `service_description`, and `end_date` for the Saturday service which has the earliest end date?

In [9]:
-- Your code here

In [10]:
-- Reveal solution
SELECT
	cal.service_id,
	cal_attrs.service_description,
	cal.end_date
FROM
	calendar
		AS cal
JOIN
	calendar_attributes
		AS cal_attrs
	ON
		cal.service_id
			= cal_attrs.service_id

WHERE
	monday
		= 0
	AND tuesday
		= 0
	AND wednesday
		= 0
	AND thursday
		= 0
	AND friday
		= 0
	AND saturday
		= 1
	AND sunday
		= 0

ORDER BY
	end_date

LIMIT
	1;

service_id,service_description,end_date
LRV420-2-Sa-01-CldCrlClgCrr,Saturday schedule,20201003


#### Which service has the most days added to the feed as exceptions?

In [11]:
-- Your code here

In [12]:
-- Reveal solution
SELECT
	service_id,
	COUNT(*)
		AS added_days
FROM
	calendar_dates
WHERE
	exception_type
		= 1

GROUP BY
	service_id

ORDER BY
	added_days
		DESC

LIMIT
	1;

service_id,added_days
RTL420-9-Wdy-01-BrrQncCnr,4


So now that we can identify which services are running trips on a given date, how can we identify *which* trips for *which* routes are running?

## Routes
*Data about individual routes*

**GTFS Table:** [routes](feed/routes.txt)

**MBTA V3 API Resource**: [Route](https://api-v3.mbta.com/docs/swagger/index.html#/Route/ApiWeb_RouteController_show)

First, let's take a look at the first few rows of the `routes` table.

In [13]:
SELECT *
FROM
	routes
LIMIT
	5;

route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order,route_fare_class,line_id,listed_route
Red,1,,Red Line,Rapid Transit,1,https://www.mbta.com/schedules/Red,DA291C,FFFFFF,10010,Rapid Transit,line-Red,
Mattapan,1,,Mattapan Trolley,Rapid Transit,0,https://www.mbta.com/schedules/Mattapan,DA291C,FFFFFF,10011,Rapid Transit,line-Mattapan,
Orange,1,,Orange Line,Rapid Transit,1,https://www.mbta.com/schedules/Orange,ED8B00,FFFFFF,10020,Rapid Transit,line-Orange,
Green-B,1,B,Green Line B,Rapid Transit,0,https://www.mbta.com/schedules/Green-B,00843D,FFFFFF,10032,Rapid Transit,line-Green,
Green-C,1,C,Green Line C,Rapid Transit,0,https://www.mbta.com/schedules/Green-C,00843D,FFFFFF,10033,Rapid Transit,line-Green,


`route_id` is a unique identifier for individual routes, which is referenced in other tables, much like `service_id` is referenced in `calendar_attributes` and `calendar_dates`.

`route_desc` categorizes a route's level of service. Possible values for the MBTA implementation include:
* `Commuter Rail`
* `Rapid Transit`
* `Local Bus`
* `Key Bus`
* `Supplemental Bus`
* `Community Bus`
* `Commuter Bus`
* `Ferry`
* `Rail Replacement Bus`

Let's look at the entry for the `89` bus.

In [14]:
SELECT *
FROM
	routes
WHERE
	route_id
		= '89';

route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order,route_fare_class,line_id,listed_route
89,1,89,Clarendon Hill or Davis Station - Sullivan Square Station,Local Bus,3,https://www.mbta.com/schedules/89,FFC72C,000000,50890,Local Bus,line-89,


### Test Your Knowledge

#### How many routes are categorized as `Local Bus`?

In [15]:
-- Your code here

In [16]:
-- Reveal solution
SELECT
	COUNT(*)
FROM
	routes
WHERE
	route_desc
		= 'Local Bus';

COUNT(*)
126


## Route Patterns
*Subsets of a route, representing different possible patterns of where trips may serve*

**GTFS Tables**: [route_patterns](feed/route_patterns.txt), [directions](feed/directions.txt)

**MBTA V3 API Resource**: [RoutePattern](https://api-v3.mbta.com/docs/swagger/index.html#/RoutePattern/ApiWeb_RoutePatternController_show)

Each individual route has one entry in the `routes` table. But you may be wondering, how do we represent the different patterns a route may take? We know a trip on a route might vary by a direction (e.g. Inbound vs Outbound) or in the pattern of stops the trip might take, like the 89 bus.

![](img/rp.png)

How do we know which patterns exist for a given route, and which pattern a given trip follows? The simplest way is to identify a given trip's `route_pattern`.

### `route_patterns`
Let's take a look at the `route_patterns` for the 89 bus.

In [17]:
SELECT *
FROM
	route_patterns
WHERE
	route_id
		= '89';

route_pattern_id,route_id,direction_id,route_pattern_name,route_pattern_time_desc,route_pattern_typicality,route_pattern_sort_order,representative_trip_id
89-2-0,89,0,Sullivan Square Station - Davis Station,,1,50890000,45771950
89-_-0,89,0,Sullivan Square Station - Clarendon Hill,Weekdays only,2,50890003,45771946
89-2-1,89,1,Davis Station - Sullivan Square Station,,1,50890100,45771952
89-_-1,89,1,Clarendon Hill - Sullivan Square Station,Weekdays only,2,50890103,45771947


Here, you can see four distinct patterns for the Orange Line:
* `89-2-0` for the Sullivan Sq to Davis Station pattern
* `89-_-0` for the Sullivan Sq to Clarendon Hill pattern
* `89-2-1` for the Davis Square Station to Sullivan Sq pattern
* `89-_-1` for the Clarendon Hill to Sullivan Sq pattern

You can also see that each pattern has a `direction_id`, either 0 or 1, representing the direction of traveled on a trip. While you could infer trip's the final destination given the `route_pattern_name`, (or a given trip's `trip_headsign`), a pattern's `direction_id` refers to an additional passenger-facing name documented in the `directions` table.

### `directions`

Let's take a look at the directions for the 89 bus.

In [18]:
SELECT *
FROM
	directions
WHERE
	route_id
		= '89';

route_id,direction_id,direction,direction_destination
89,0,Outbound,Clarendon Hill or Davis Station
89,1,Inbound,Sullivan Square Station


Here, we can see that the `89-2-0` and `89-_-0` patterns could also be refered to as "Outbound" and the `89-2-1` and `89-_-1` patterns could also be refered to as "Inbound."

With the combination of `service_id`, `route_id`, `route_pattern_id`, and `direction_id`, we can tell which routes are running which trips on a given date.

### Test Your Knowledge

#### Which routes have exactly two patterns?

In [19]:
-- Your code here

In [20]:
-- Reveal solution
SELECT
	route_id,
	COUNT(*)
		AS n_patterns
FROM
	route_patterns
GROUP BY
	route_id
HAVING
	n_patterns
		= 2;

route_id,n_patterns
1,2
104,2
105,2
109,2
114,2
116117,2
132,2
137,2
17,2
18,2


## Trips
*The journey of a particular vehicle through a set of stops*

**GTFS Table**: [trips](feed/trips.txt)

**MBTA V3 API Resource**: [`Trip`](https://api-v3.mbta.com/docs/swagger/index.html#/Trip/ApiWeb_TripController_show)

The `trips` table contains data for each individual trip in a GTFS feed. Let's look at the first few rows.

In [21]:
SELECT *
FROM
	trips
LIMIT
	3;

route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed
1,ColumbusDay-VeteransDay-DayafterThanksgivingDay,45692465,Harvard,,0,C01-9,010085,1,,1-_-0,1
1,ColumbusDay-VeteransDay-DayafterThanksgivingDay,45692467,Harvard,,0,C01-9,010085,1,,1-_-0,1
1,ColumbusDay-VeteransDay-DayafterThanksgivingDay,45692469,Harvard,,0,C01-14,010085,1,,1-_-0,1


Once again, we see a `service_id` column corresponding to a row in the `calendar` table.

Additionally, we see:
* `trip_id`: a unique identifier for the trip
* `route_id`: a unique identifier for a route in the `routes` table, which contains data about each route
* `route_pattern_id`: a unique identifier for a pattern in the `route_patterns` table, which contains data about each combination of stops a route may take

You may also notice there is no date field. That is because in GTFS, a `trip` is not an individual trip as a rider would perceive it (i.e. “the Inbound 89 trip that occurs at 8:39am on October 27th, 2020”), but a trip that occurs once on every date defined for a given service. 

Let's take a look at all of the services which include trips by the 89 bus.

In [22]:
SELECT DISTINCT
	service_id
FROM
	trips
WHERE
	route_id
		= '89';

service_id
ColumbusDay-VeteransDay-DayafterThanksgivingDay
FallSaturday
FallSunday
FallWeekday
ThanksgivingDay


As a reminder, these services can include many trips from many different routes.

In [23]:
SELECT
	COUNT(*)
FROM (
	SELECT DISTINCT
		route_id
	FROM
		trips
	WHERE
		service_id
			== 'FallWeekday'
);

COUNT(*)
154


Now, let's take a look at all of the trips for the 89 bus on the `FallWeekday` service.

In [24]:
SELECT *
FROM
	trips
WHERE
	service_id
		== 'FallWeekday'
	AND route_id
		= '89';

route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed
89,FallWeekday,45771950,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771953,Davis,,0,G89-3,890135,1,,89-2-0,1
89,FallWeekday,45771954,Davis,,0,G89-3,890135,1,,89-2-0,1
89,FallWeekday,45771958,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771960,Davis,,0,G89-3,890135,1,,89-2-0,1
89,FallWeekday,45771964,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771966,Davis,,0,G89-2,890135,1,,89-2-0,1
89,FallWeekday,45771977,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771979,Davis,,0,G89-2,890135,1,,89-2-0,1
89,FallWeekday,45771982,Davis,,0,G89-14,890135,1,,89-2-0,1


Finally, let's look at only trips on the Sullivan Sq to Davis Square pattern (recall this is `89-2-0` as seen above).

In [25]:
SELECT *
FROM
	trips
WHERE
	service_id
		== 'FallWeekday'
	AND route_pattern_id
		= '89-2-0';

route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,trip_route_type,route_pattern_id,bikes_allowed
89,FallWeekday,45771950,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771953,Davis,,0,G89-3,890135,1,,89-2-0,1
89,FallWeekday,45771954,Davis,,0,G89-3,890135,1,,89-2-0,1
89,FallWeekday,45771958,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771960,Davis,,0,G89-3,890135,1,,89-2-0,1
89,FallWeekday,45771964,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771966,Davis,,0,G89-2,890135,1,,89-2-0,1
89,FallWeekday,45771977,Davis,,0,G89-6,890135,1,,89-2-0,1
89,FallWeekday,45771979,Davis,,0,G89-2,890135,1,,89-2-0,1
89,FallWeekday,45771982,Davis,,0,G89-14,890135,1,,89-2-0,1


### Test Your Knowledge

#### For each service running ***89*** trips, how many trips are on the `Sullivan Sq to Clarendon Hill` pattern?

In [26]:
-- Your code here

In [27]:
-- Reveal solution
SELECT
	service_id,
	COUNT(*)
		AS n_trips
FROM
	TRIPS
WHERE
	route_pattern_id
		= '89-_-0'
GROUP BY
	service_id;

service_id,n_trips
ColumbusDay-VeteransDay-DayafterThanksgivingDay,18
FallWeekday,18


## Schedules
*When a vehicle should arrive and depart a given stop for a given trip*

**GTFS Table**: [`stop_times`](feed/stop_times.txt)

**MBTA V3 API Resource**: [`Schedule`](https://api-v3.mbta.com/docs/swagger/index.html#/Schedule/ApiWeb_ScheduleController_index)

The `stop_times` table includes a row for every scheduled stop on every scheduled trip in a GTFS Feed. If that sounds like a lot of rows, it is! This is by far the largest table in the feed. Here, we can see there are `2,019,434` `stop_times` in this version of the feed.

In [28]:
SELECT
	COUNT(*)
FROM
	stop_times;

COUNT(*)
2019434


Let's take a look at the first few rows.

In [29]:
SELECT *
FROM
	stop_times
LIMIT
	10;

trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,continuous_pickup,continuous_drop_off
45295219,05:16:00,05:16:00,70061,1,,0,1,0,alfcl,,
45295219,05:18:00,05:18:00,70063,10,,0,0,0,davis,,
45295219,05:20:00,05:20:00,70065,20,,0,0,0,portr,,
45295219,05:23:00,05:23:00,70067,30,,0,0,0,harsq,,
45295219,05:27:00,05:27:00,70069,40,,0,0,0,cntsq,,
45295219,05:29:00,05:29:00,70071,50,,0,0,0,knncl,,
45295219,05:31:00,05:31:00,70073,60,,0,0,0,chmnl,,
45295219,05:33:00,05:33:00,70075,70,,0,0,0,pktrm,,
45295219,05:34:00,05:34:00,70077,80,,0,0,0,dwnxg,,
45295219,05:36:00,05:36:00,70079,90,,0,0,0,sstat,,


Each `trip_id` corresponds to a row in the `trips` table, and there should be one row for every stop that trip makes.

The `arrival_time` and `departure_time` indicate when a vehicle should arrive and depart the stop identified by `stop_id` (more on stops later).

The `stop_sequence` field indicates the order of stops for a particular trip. The values must increase along the trip but do not need to be consecutive.

So if we have a given `trip_id`, we can see all of the stops in order for that trip.

In [30]:
SELECT *
FROM
	stop_times
WHERE
	trip_id
		= '45295219'
ORDER BY
	stop_sequence;

trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,timepoint,checkpoint_id,continuous_pickup,continuous_drop_off
45295219,05:16:00,05:16:00,70061,1,,0,1,0,alfcl,,
45295219,05:18:00,05:18:00,70063,10,,0,0,0,davis,,
45295219,05:20:00,05:20:00,70065,20,,0,0,0,portr,,
45295219,05:23:00,05:23:00,70067,30,,0,0,0,harsq,,
45295219,05:27:00,05:27:00,70069,40,,0,0,0,cntsq,,
45295219,05:29:00,05:29:00,70071,50,,0,0,0,knncl,,
45295219,05:31:00,05:31:00,70073,60,,0,0,0,chmnl,,
45295219,05:33:00,05:33:00,70075,70,,0,0,0,pktrm,,
45295219,05:34:00,05:34:00,70077,80,,0,0,0,dwnxg,,
45295219,05:36:00,05:36:00,70079,90,,0,0,0,sstat,,


Data about the physical stops referenced by `stop_id` can be found in the `stops` table.

### Test Your Knowledge

#### At what time should the trip `45692465` arrive at its final stop?

In [31]:
-- Your code here

In [32]:
-- Reveal solution
SELECT
	arrival_time
FROM
	stop_times
WHERE
	trip_id
		= '45692465'
	AND stop_sequence
		= (
			SELECT
				MAX(stop_sequence)
			FROM
				stop_times
			WHERE
				trip_id
					= '45692465'
			GROUP BY
				trip_id
		);

arrival_time
05:00:00


## Stops
*Details for a specific stop*

**GTFS Table**: [`stops`](feed/stops.txt)

**MBTA V3 API Resource**: [`Stop`](https://api-v3.mbta.com/docs/swagger/index.html#/Stop/ApiWeb_StopController_show)

Let's take a look at the first few rows of the `stops` table.

In [33]:
SELECT *
FROM
	stops
LIMIT
	10;

stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type
1,1,Washington St opp Ruggles St,,,,42.330957,-71.082754,ExpressBus-Downtown,,https://www.mbta.com/stops/1,,0,,1,Boston,Washington Street,Ruggles Street,3
10,10,Theo Glynn Way @ Newmarket Sq,,,,42.330555,-71.068787,LocalBus,,https://www.mbta.com/stops/10,,0,,1,Boston,Theodore Glynn Way,Newmarket Square,3
10000,10000,Tremont St opp Temple Pl,,,,42.355692,-71.062911,LocalBus,,https://www.mbta.com/stops/10000,,0,,1,Boston,Tremont Street,Temple Place,3
10003,10003,Albany St opp Randall St,,,,42.331591,-71.076237,LocalBus,,https://www.mbta.com/stops/10003,,0,,0,Boston,Albany Street,Randall Street,3
10005,10005,Albany St opp E Concord St,,,,42.335017,-71.071280,LocalBus,,https://www.mbta.com/stops/10005,,0,,1,Boston,Albany Street,East Concord Street,3
10006,10006,Albany St opp E Newton St,,,,42.335814,-71.070135,LocalBus,,https://www.mbta.com/stops/10006,,0,,1,Boston,Albany Street,East Newton Street,3
10007,10007,540 Albany St opp Wareham St,,,,42.338372,-71.066556,LocalBus,,https://www.mbta.com/stops/10007,,0,,1,Boston,Albany Street,Wareham Street,3
10008,10008,Albany St opp Randolph St,,,,42.339915,-71.064356,LocalBus,,https://www.mbta.com/stops/10008,,0,,0,Boston,Albany Street,,3
10009,10009,High St @ Duck Lane,,,,42.214986,-70.915924,LocalBus,,https://www.mbta.com/stops/10009,,0,,0,Hingham,High Street,,3
1001,1001,Newton St @ Arlington Rd,,,,42.307661,-71.169522,LocalBus,,https://www.mbta.com/stops/1001,,0,,0,Brookline,Newton Street,,3


This information can be joined with the `stop_times` table to construct a readable schedule for a given trip.

In [34]:
SELECT
	stop_time.trip_id
		AS trip_id,
	stop_time.arrival_time
		AS arrival_time,
	stop_time.departure_time
		AS departure_time,
	stop_time.stop_id
		AS stop_id,
	stop_time.stop_sequence
		AS stop_sequence,
	stop.stop_name
		AS stop_name
FROM
	stop_times
		AS stop_time
JOIN
	stops
		AS stop
	ON
		stop_time.stop_id
			= stop.stop_id
WHERE
	trip_id
		= '45771950'
ORDER BY
	stop_sequence;

trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name
45771950,06:07:00,06:07:00,29011,1,Sullivan Square
45771950,06:10:00,06:10:00,2718,2,Broadway @ Austin St
45771950,06:10:00,06:10:00,2719,3,Broadway @ Indiana Ave
45771950,06:11:00,06:11:00,2720,4,Broadway @ Michigan Ave
45771950,06:11:00,06:11:00,2721,5,Broadway @ Cross St
45771950,06:12:00,06:12:00,2722,6,Broadway @ Kensington Ave
45771950,06:13:00,06:13:00,2723,7,Broadway @ Fellsway W
45771950,06:14:00,06:14:00,2725,8,Broadway @ Temple St
45771950,06:14:00,06:14:00,2726,9,Broadway @ Winter Hill Plaza
45771950,06:15:00,06:15:00,2729,10,Broadway @ Main St


### Test Your Knowledge

#### Display the following fields for the last trip of the day for the ***89*** bus on the ***Sullivan Sq to Clarendon Hill*** pattern on ***Fall weekdays***:
* `stop_id`
* `stop_name`
* `arrival_time`
* `departure_time`

In [35]:
-- Your code here

In [36]:
-- Reveal solution

SELECT
	stop_time.stop_id
		AS stop_id,
	stop.stop_name
		AS stop_name,
	MAX(stop_time.arrival_time)
		AS arrival_time,
	MAX(stop_time.departure_time)
		AS departure_time
FROM
	stop_times
		AS stop_time
JOIN
	stops
		AS stop
	ON
		stop_time.stop_id
		== stop.stop_id
JOIN
	trips
		AS trip
	ON
		stop_time.trip_id
		== trip.trip_id

WHERE
	trip.route_id
		== '89'
	AND trip.service_id
		== 'FallWeekday'
	AND trip.route_pattern_id
		== '89-_-1'

GROUP BY
	stop_time.stop_sequence,
	stop.stop_name

ORDER BY
	stop_sequence

stop_id,stop_name,arrival_time,departure_time
2637,Clarendon Hill Busway,19:02:00,19:02:00
2575,Broadway @ Garrison Ave,19:03:00,19:03:00
2576,Broadway @ Weston Ave,19:03:00,19:03:00
2691,Broadway opp Packard Ave,19:04:00,19:04:00
2692,Broadway @ Simpson Ave,19:05:00,19:05:00
2693,Broadway @ Billingham St,19:06:00,19:06:00
2694,Broadway @ College Ave,19:07:00,19:07:00
2695,Broadway opp Warner St - Powder House Sq,19:07:00,19:07:00
2696,Broadway @ Bay State Ave,19:07:00,19:07:00
2697,Broadway @ Josephine Ave - Ball Sq,19:08:00,19:08:00


## What's next?

Open up a [new notebook](GTFS_Exploration.ipynb) and explore the current feed!